In [ ]:
import numpy as np
import scipy.linalg as la

In [33]:
def gram_schmidt_orthogonalization(n):
    X, W = np.polynomial.legendre.leggauss(n)
    V = np.vander(X, increasing=True)
    G = V.T @ np.diag(W) @ V
    Q = la.cholesky(G, lower=True)
    Q_inv = la.inv(Q)
    ortho_polys = [np.poly1d(Q_inv[i, ::-1]) for i in range(n)]
    return ortho_polys

def recurrence_relation(n):
    polys = [np.poly1d([1])]
    if n > 1:
        polys.append(np.poly1d([1, 0]))
    for k in range(2, n):
        Pk = ((2*k - 1) * np.poly1d([1, 0]) * polys[k-1] - (k - 1) * polys[k-2]) / k
        polys.append(Pk)
    return polys

def check_orthogonality(polys, n):
    X, W = np.polynomial.legendre.leggauss(n)
    V = np.array([[p(x) for p in polys] for x in X])
    G = V.T @ np.diag(W) @ V
    return np.allclose(G - np.diag(np.diag(G)), 0)

n = 10
gs_polys = gram_schmidt_orthogonalization(n)
rec_polys = recurrence_relation(n)

print("Gram-Schmidt:")
print("Orthogonality:", check_orthogonality(gs_polys, n))

print()

print("Recurrent relations:")
print("Orthogonality:", check_orthogonality(rec_polys, n))

Gram-Schmidt:
Orthogonality: True

Recurrent relations:
Orthogonality: True


In [2]:
import numpy as np
import scipy.linalg as la

def create_jacobi_matrix(n, alpha=1e-9, beta=1e-9):
    J = np.zeros((n, n))
    for i in range(n - 1):
        J[i, i + 1] = np.sqrt((i + 1) * (i + alpha) / (2 * (i + beta)))
        J[i + 1, i] = J[i, i + 1]
    for i in range(n):
        J[i, i] = (2 * i + alpha + beta)
    return J

def orthogonal_polynomials_from_jacobi_matrix(jacobi_matrix):
    eigenvalues, eigenvectors = la.eig(jacobi_matrix)
    eigenvalues = eigenvalues.real
    eigenvectors = eigenvectors.real
    norm_eigenvectors = np.zeros_like(eigenvectors)
    
    for i in range(eigenvectors.shape[1]):
        norm_eigenvectors[:, i] = eigenvectors[:, i] / np.linalg.norm(eigenvectors[:, i])

    polynomials = []
    for i in range(norm_eigenvectors.shape[1]):
        polynomials.append(norm_eigenvectors[:, i])
    
    return polynomials

def check_orthogonality(polynomials, epsilon=1e-10):
    n = len(polynomials)
    for i in range(n):
        for j in range(i + 1, n):
            dot_product = np.dot(polynomials[i], polynomials[j])
            if abs(dot_product) > epsilon:
                print(f"Polynomials {i+1} and {j+1} not orthogonal! Dot product: {dot_product}")
                return False
    return True

n = 10
jacobi_matrix = create_jacobi_matrix(n)
polynomials = orthogonal_polynomials_from_jacobi_matrix(jacobi_matrix)
print("Orthogonal:", check_orthogonality(polynomials))


Orthogonal: True


In [109]:
from scipy import integrate
import numpy as np


# Polynomials count in the system
# For simplicity max degree of the polynomial in the system is equal to its size 
segment_a = -1
segment_b = 1


def jacobi_weight_function(x):
    return ((1 - x) ** 2) * ((1 + x) ** 2)


def calc_polynom(x, pol):
    res = 0
    for i in range(len(pol)):
        res += pol[i] * x**i
    return res


def f_g_with_weight(x, f_pol, g_pol):
    w = jacobi_weight_function(x)
    f = calc_polynom(x, f_pol)
    g = calc_polynom(x, g_pol)
    return f * g * w


def dot_product(f_pol, g_pol):
    res, _ = integrate.quad(f_g_with_weight, segment_a, segment_b, args=(f_pol, g_pol))
    return res


def is_orthogonal(f_pol, g_pol, epsilon: float) -> bool:
    return np.allclose(dot_product(f_pol, g_pol), 0, atol=epsilon)


def is_orthogonal_system(polynomials, epsilon: float) -> bool:
    for i, f_pol in enumerate(polynomials):
        for j, g_pol in enumerate(polynomials[:i]):
            if not is_orthogonal(f_pol, g_pol, epsilon):
                print(f'Polynomials {i} and {j} are not orthogonal!')
                return False
    return True


In [151]:
# Find orthogonal polynomials using Gram-Schmidt

# For polynomial degrees greater than 10-15 we get huge numerical errors
pol_system_size = 5
epsilon = 1e-6

from scipy.linalg import cholesky

unit_polynomials = np.eye(pol_system_size)

def build_gs_matrix(n: int) -> np.array:
    gs_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            gs_matrix[i][j] = dot_product(unit_polynomials[i], unit_polynomials[j])
    return gs_matrix
     
gs_matrix = build_gs_matrix(pol_system_size)

U = cholesky(gs_matrix)

polynomials = np.linalg.inv(U).transpose()

print(polynomials)
print(is_orthogonal_system(polynomials, epsilon))



[[  0.96824584   0.           0.           0.           0.        ]
 [  0.           2.56173769   0.           0.           0.        ]
 [ -0.83852549   0.           5.86967844   0.           0.        ]
 [  0.          -4.24816137   0.          12.7444841    0.        ]
 [  0.8163969    0.         -14.69514429   0.          26.94109786]]
True


In [ ]:
# Build orthogonal polynomials using recurrence relations
n = 30

def polynomial_add(p1, p2):
    """Add two polynomials p1 + p2"""
    max_len = max(len(p1), len(p2))
    result = np.zeros(max_len)
    
    for i in range(max_len):
        coeff1 = p1[i] if i < len(p1) else 0
        coeff2 = p2[i] if i < len(p2) else 0
        result[i] = coeff1 + coeff2
    
    return result


def polynomial_multiply_scalar(p, scalar):
    """Multiply polynomial by scalar"""
    return np.array(p) * scalar


def polynomial_multiply_x(p):
    """Multiply polynomial by x (shift coefficients)"""
    if len(p) == 0:
        return np.array([0])
    
    result = np.zeros(len(p) + 1)
    result[1:] = p  # shift all coefficients to the right
    return result


def polynomial_multiply_by_linear(poly, linear):
    """Multiply polynomial poly by linear polynomial linear = [a, b] (represents a + bx)"""
    if len(poly) == 0 or len(linear) != 2:
        return np.array([])
    
    a, b = linear[0], linear[1]
    result = np.zeros(len(poly) + 1)
    
    # poly * a
    for i in range(len(poly)):
        result[i] += poly[i] * a
    
    # poly * (b * x)
    for i in range(len(poly)):
        result[i + 1] += poly[i] * b
    
    return result


def polynomial_subtract(p1, p2):
    """Subtract polynomials p1 - p2"""
    max_len = max(len(p1), len(p2))
    result = np.zeros(max_len)
    
    for i in range(max_len):
        coeff1 = p1[i] if i < len(p1) else 0
        coeff2 = p2[i] if i < len(p2) else 0
        result[i] = coeff1 - coeff2
    
    return result


def build_recurrence_polynomials(n):
    """
    Build orthogonal polynomials using recurrence relations
    
    Recurrence formula:
    P_{n+1}(x) = (x - α_n) * P_n(x) - β_n * P_{n-1}(x)
    
    where:
    α_n = ⟨x*P_n, P_n⟩ / ⟨P_n, P_n⟩
    β_n = ⟨P_n, P_n⟩ / ⟨P_{n-1}, P_{n-1}⟩
    """
    print("Build orthogonal polynomials using recurrence relations")
    print("=" * 70)
    
    polynomials = []
    
    # P_0(x) = 1 (normalized)
    P_0 = np.array([1 if j == 0 else 0 for j in range(n)])
    P_0_norm_squared = dot_product(P_0, P_0)
    P_0_normalized = polynomial_multiply_scalar(P_0, 1.0 / np.sqrt(P_0_norm_squared))
    polynomials.append(P_0_normalized)
    
    if n == 1:
        return polynomials
    
    # P_1(x) = x - α_0, где α_0 = ⟨x*P_0, P_0⟩ / ⟨P_0, P_0⟩
    x_poly = np.array([1 if j == 1 else 0 for j in range(n)])  # representation of the polynomial x
    
    # Compute x * P_0
    x_P_0 = polynomial_multiply_x(P_0_normalized)
    
    # α_0 = ⟨x*P_0, P_0⟩ / ⟨P_0, P_0⟩
    numerator = dot_product(x_P_0, P_0_normalized)
    denominator = dot_product(P_0_normalized, P_0_normalized)
    alpha_0 = numerator / denominator
    
    # P_1 = x - α_0
    P_1 = polynomial_add(x_poly, polynomial_multiply_scalar(np.array([1 if j == 0 else 0 for j in range(n)]), -alpha_0))
    
    # Normalize P_1
    P_1_norm_squared = dot_product(P_1, P_1)
    P_1_normalized = polynomial_multiply_scalar(P_1, 1.0 / np.sqrt(P_1_norm_squared))
    polynomials.append(P_1_normalized)
    
    # Build P_2, P_3, ..., P_{n-1} recursively
    for k in range(2, n):
        
        P_k_minus_1 = polynomials[k-1]  # P_{k-1}
        P_k_minus_2 = polynomials[k-2]  # P_{k-2}

        # Compute x * P_{k-1}
        x_P_k_minus_1 = polynomial_multiply_x(P_k_minus_1)
        
        # α_{k-1} = ⟨x*P_{k-1}, P_{k-1}⟩
        alpha_k_minus_1 = dot_product(x_P_k_minus_1, P_k_minus_1)
        
        # β_{k-1} * P_k(x) = (x - α_{k-1}) * P_{k-1}(x) - β_{k-2} * P_{k-2}(x)
        first_term = polynomial_subtract(x_P_k_minus_1, polynomial_multiply_scalar(P_k_minus_1, alpha_k_minus_1))

        beta_k_minus_2 = P_k_minus_2[k-2] / P_k_minus_1[k-1]
        second_term = polynomial_multiply_scalar(P_k_minus_2, beta_k_minus_2)

        P_k = polynomial_subtract(first_term, second_term)

        # Normalize P_k
        P_k_norm_squared = dot_product(P_k, P_k)
        P_k_normalized = polynomial_multiply_scalar(P_k, 1.0 / np.sqrt(P_k_norm_squared))
        polynomials.append(P_k_normalized)

    return polynomials

print(f"Building a system of {n} orthogonal polynomials")
print(f"Method: Recurrence relations")
print(f"Interval: [{segment_a}, {segment_b}]")
print(f"Weight function: w(x) = (1-x)^(3/2) * (1+x)^2")
print("=" * 70)

# Build orthogonal system
orthogonal_system = build_recurrence_polynomials(n)

# Check orthogonality
print("\n" + "=" * 70)
print("CHECK ORTHOGONALITY:")
print("=" * 70)

epsilon = 1e-6
is_orthog = is_orthogonal_system(orthogonal_system, epsilon)

if is_orthog:
    print("✓ System of polynomials is orthogonal!")
else:
    print("✗ System of polynomials is NOT orthogonal!")

Building a system of 30 orthogonal polynomials
Method: Recurrence relations
Interval: [-1, 1]
Weight function: w(x) = (1-x)^(3/2) * (1+x)^2
Build orthogonal polynomials using recurrence relations

CHECK ORTHOGONALITY:
✓ System of polynomials is orthogonal!


In [186]:
# Build orthogonal polynomials using matrix eigenvalues
import scipy.linalg

def build_polynomials_with_eigenvalues(n):
    """
    Build orthogonal polynomials using matrix eigenvalues
    
    Recurrence formula:
    P_{n+1}(x) = (x - α_n) * P_n(x) - β_n * P_{n-1}(x)
    
    where:
    α_n = ⟨x*P_n, P_n⟩ / ⟨P_n, P_n⟩
    β_n = ⟨P_n, P_n⟩ / ⟨P_{n-1}, P_{n-1}⟩
    """
    print("Build orthogonal polynomials using matrix eigenvalues")
    print("=" * 70)
    
    matrix = np.zeros((n, n))
    polynomials = []
    
    # P_0(x) = 1 (normalized)
    P_0 = np.array([1 if j == 0 else 0 for j in range(n)])
    P_0_norm_squared = dot_product(P_0, P_0)
    P_0_normalized = polynomial_multiply_scalar(P_0, 1.0 / np.sqrt(P_0_norm_squared))
    polynomials.append(P_0_normalized)
    
    if n == 1:
        return polynomials
    
    # P_1(x) = x - α_0, где α_0 = ⟨x*P_0, P_0⟩ / ⟨P_0, P_0⟩
    x_poly = np.array([1 if j == 1 else 0 for j in range(n)])  # representation of the polynomial x
    
    # Compute x * P_0
    x_P_0 = polynomial_multiply_x(P_0_normalized)
    
    # α_0 = ⟨x*P_0, P_0⟩ / ⟨P_0, P_0⟩
    numerator = dot_product(x_P_0, P_0_normalized)
    denominator = dot_product(P_0_normalized, P_0_normalized)
    alpha_0 = numerator / denominator
    matrix[0][0] = alpha_0
    
    # P_1 = x - α_0
    P_1 = polynomial_add(x_poly, polynomial_multiply_scalar(np.array([1 if j == 0 else 0 for j in range(n)]), -alpha_0))
    
    # Normalize P_1
    P_1_norm_squared = dot_product(P_1, P_1)
    P_1_normalized = polynomial_multiply_scalar(P_1, 1.0 / np.sqrt(P_1_norm_squared))
    polynomials.append(P_1_normalized)

    # Build P_2, P_3, ..., P_{n-1} with matrix eigenvalues
    for k in range(2, n):
        
        P_k_minus_1 = polynomials[k-1]  # P_{k-1}
        P_k_minus_2 = polynomials[k-2]  # P_{k-2}
        
        x_P_k_minus_1 = polynomial_multiply_x(P_k_minus_1)
        # α_{k-1} = ⟨x*P_{k-1}, P_{k-1}⟩
        alpha_k_minus_1 = dot_product(x_P_k_minus_1, P_k_minus_1)
        matrix[k-1][k-1] = alpha_k_minus_1

        beta_k_minus_2 = P_k_minus_2[k-2] / P_k_minus_1[k-1]
        matrix[k-1][k-2] = beta_k_minus_2
        matrix[k-2][k-1] = beta_k_minus_2

        diag = np.diag(matrix[:k, :k])[:k]
        off_diag = np.diag(matrix[:k, :k], k=1)[:k-1]
        roots = scipy.linalg.eigh_tridiagonal(diag, off_diag, eigvals_only=True)[:k]

        P_k = np.poly(roots)[::-1]

        # Normalize P_k
        P_k_norm_squared = dot_product(P_k, P_k)
        P_k_normalized = polynomial_multiply_scalar(P_k, 1.0 / np.sqrt(P_k_norm_squared))
        polynomials.append(P_k_normalized)

    return polynomials

n = 15

# Build orthogonal system
polys = build_polynomials_with_eigenvalues(n)

# Check orthogonality
print("\n" + "=" * 70)
print("CHECK ORTHOGONALITY:")
print("=" * 70)

epsilon = 1e-6
is_orthog = is_orthogonal_system(polys, epsilon)

if is_orthog:
    print("✓ System of polynomials is orthogonal!")
else:
    print("✗ System of polynomials is NOT orthogonal!")



Build orthogonal polynomials using matrix eigenvalues

CHECK ORTHOGONALITY:
✓ System of polynomials is orthogonal!
